## Integrate radially first

Important: Hyperspy file needs to be calibrated and distortion corrected and beam centered

In [1]:
import warnings

from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
%matplotlib qt
import pyxem as pxm
import hyperspy.api as hs
import os, glob, tqdm
import gc
import numpy as np
from diffsims.utils.sim_utils import get_electron_wavelength

In [ ]:
root = r'G:\My Drive\PhD\projects\external_measurements\ml_difsims'
folder = 'data/experimental'
file_extension = '*.hspy'

path = os.path.join(root, folder, file_extension)
paths = glob.glob(path)
paths = [p for p in paths if 'radial' not in p]
paths.sort()
paths

In [3]:
from pyxem.detectors import Medipix515x515Detector

# Simulation microscope values (for azimuthal integration)
beam_energy = 200.0  #keV
detector_pix_size = 55e-6  #m

for p in tqdm.tqdm(paths[1:2]):
    print(p)
    dp = hs.load(p, signal_type='electron_diffraction')

    calibration = dp.axes_manager.signal_axes[0].scale
    detector_size = dp.axes_manager.signal_axes[0].size
    radial_steps = int(np.ceil((int(detector_size / 2) - 1) / 2) * 2)

    # # Old way (pyxem way):
    #dp.set_diffraction_calibration(calibration)
    dp.metadata.Signal.ai = None
    dp.unit = "k_A^-1"
    dp.set_experimental_parameters(beam_energy=beam_energy)
    dp.set_ai(center=([detector_size / 2, detector_size / 2]))

    # New way (pyFAI)
    # detector = Medipix515x515Detector()
    # wavelength = get_electron_wavelength(beam_energy) * 1e-10
    # camera_length = detector_pix_size / (wavelength * calibration * 1e10)
    # center = ([detector_size / 2, detector_size / 2])
    # unit = "k_A^-1"
    # dp.unit = unit
    # ai = AzimuthalIntegrator(dist=camera_length, detector=detector, wavelength=wavelength)
    # ai.setFit2D(directDist=camera_length * 1000, centerX=center[1], centerY=center[0])
    # dp.metadata.set_item("Signal.ai", ai)

    radial = dp.get_azimuthal_integral1d(npt=radial_steps)
    radial2d = dp.get_azimuthal_integral2d(npt=radial_steps)

    print(radial)
    name = "{}_radial.hspy".format(os.path.basename(p).split('.')[0])
    radial.save(os.path.join(os.path.dirname(p), name), overwrite=True)
    name = "{}_radial2d.hspy".format(os.path.basename(p).split('.')[0])
    radial2d.save(os.path.join(os.path.dirname(p), name), overwrite=True)
    print(f"Finished with {name}")
    gc.collect()


  0%|          | 0/1 [00:00<?, ?it/s]

G:\My Drive\PhD\projects\external_measurements\ml_difsims\data/experimental\20220126_142402_rebin_nav_2.hspy


ERROR:hyperspy.io:If this file format is supported, please report this error to the HyperSpy developers.
  0%|          | 0/1 [00:16<?, ?it/s]


KeyboardInterrupt: 

## Crop (and rebin) to match simulated range

The simulated range has the following format. Make sure the processed exp data has the same:
- Pixel size: 147 px
- Range in q (no $2\pi/d$ but just $1/d$): (0.10777668889613681, 1.318191810345058) $\AA^{-1}$

In [4]:
# DO NOT CHANGE!!!
crop_range_q = (0.10777668889613681, 1.318191810345058) # A-1
crop_size = 147 #px

In [5]:
%matplotlib qt
import hyperspy.api as hs
import os, glob
import numpy as np
from scipy import interpolate
import warnings
import matplotlib.pyplot as plt

In [14]:
def interpolate_1d(signal_data, q_array, crop_range_q, crop_size):
    # Do interpolation
    x = q_array
    y = signal_data
    f = interpolate.interp1d(x, y, fill_value='extrapolate')

    # Generate new data
    x_new = np.linspace(crop_range_q[0], crop_range_q[1], crop_size)
    y_interpol = f(x_new)
    return y_interpol

def interpolate_2d(signal_data, q_array, crop_range_q, crop_size):
    signal_data = signal_data.T

    y_interpol_2d = \
        np.vstack([interpolate_1d(row, q_array, crop_range_q, crop_size)
                   for row in signal_data])
    return y_interpol_2d.T

In [8]:
root = r'G:\My Drive\PhD\projects\external_measurements\ml_difsims'
folder = 'data/experimental'
file_extension = '*.hspy'

path = os.path.join(root, folder, file_extension)
paths = glob.glob(path)
paths = [p for p in paths if 'radial.' in p and 'crop' not in p]
paths.sort()
paths

['G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\20200209_163154_centre_rebin_correct_rb_fullscan_radial.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\20220126_142402_rebin_nav_2_radial.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\roi_3_rebin_radial.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\roi_4_rebin_radial.hspy']

In [8]:
for path in paths:
    dp = hs.load(path, signal_type='electron_diffraction')
    q_exp = dp.axes_manager.signal_axes[0].axis
    if q_exp.min() > crop_range_q[0] or q_exp.max() < crop_range_q[1]:
        warnings.warn("The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d")

    q_array = dp.axes_manager.signal_axes[0].axis
    q_new = np.linspace(crop_range_q[0], crop_range_q[1], crop_size)
    dp_crop = dp.map(interpolate_1d, q_array = q_array, crop_range_q = crop_range_q, crop_size = crop_size,
                     show_progressbar=True, parallel=True, inplace=False)

    # Correct for axes calibration
    sig_ax = dp_crop.axes_manager.signal_axes[0]
    sig_ax.offset = crop_range_q[0]
    sig_ax.scale = (crop_range_q[1] - crop_range_q[0])/ crop_size
    print(dp_crop)

    # Save files
    name = "{}_crop.hspy".format(os.path.basename(path).split('.')[0])
    dp_crop.save(os.path.join(os.path.dirname(path), name), overwrite=True)
    name = "{}_crop.npz".format(os.path.basename(path).split('.')[0])
    np.savez(os.path.join(os.path.dirname(path), 'npz_files', name), y=dp_crop.data, x=q_new)

    # Plot results
    plt.plot(q_new, dp_crop.mean().data, label=name)

plt.legend()
plt.tight_layout()

<ElectronDiffraction2D, title: , dimensions: (127, 127|147, 160)>


ValueError: x and y must have same first dimension, but have shapes (147,) and (160, 147)

In [36]:


# Get the experimental q range


(160,)
(360, 160)


C:\Users\jf631\AppData\Local\Temp\ipykernel_13868\3885628504.py:16: UserWarning: The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d
  warnings.warn(


# Interpolate 2D

In [11]:
root = r'G:\My Drive\PhD\projects\external_measurements\ml_difsims'
folder = 'data/experimental'
file_extension = '*.hspy'

path = os.path.join(root, folder, file_extension)
paths = glob.glob(path)
paths_2d = [p for p in paths if 'radial2d.' in p and 'crop' not in p]
paths_2d.sort()
paths_2d

['G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\20220126_142402_rebin_nav_2_radial2d.hspy']

In [15]:
for path in paths_2d:
    dp = hs.load(path, signal_type='electron_diffraction')

    q_exp = dp.axes_manager.signal_axes[1].axis
    if q_exp.min() > crop_range_q[0] or q_exp.max() < crop_range_q[1]:
        warnings.warn(
            "The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d")
    q_new = np.linspace(crop_range_q[0], crop_range_q[1], crop_size)

    dp_crop = dp.map(interpolate_2d, q_array = q_exp, crop_range_q = crop_range_q, crop_size = crop_size,
                     show_progressbar=True, parallel=True, inplace=False)

    # Correct for axes calibration
    sig_ax = dp_crop.axes_manager.signal_axes[1]
    sig_ax.offset = crop_range_q[0]
    sig_ax.scale = (crop_range_q[1] - crop_range_q[0]) / crop_size

    # Normalise data cropped for the npz
    dpmax = dp_crop.data.max((-2, -1), keepdims=True)
    dpmin = dp_crop.data.min((-2, -1), keepdims=True)
    dp_crop_dat = (dp_crop.data - dpmin) / (dpmax - dpmin)
    # Correct any nan value
    nan_mask = np.isnan(dp_crop_dat)
    dp_crop_dat[nan_mask] = 0

    # Save files
    name = "{}_crop2d.hspy".format(os.path.basename(path).split('.')[0])
    dp_crop.save(os.path.join(os.path.dirname(path), name), overwrite=True)
    name = "{}_crop2d.npz".format(os.path.basename(path).split('.')[0])
    np.savez(os.path.join(os.path.dirname(path), 'npz_files', name), y=dp_crop_dat, x=q_new)

    # Plot results
    plt.plot(q_new, dp_crop.mean().data, label=name)

plt.legend()
plt.tight_layout()

C:\Users\jf631\AppData\Local\Temp\ipykernel_11040\499320023.py:6: UserWarning: The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d
  warnings.warn(
C:\Users\jf631\AppData\Local\Temp\ipykernel_11040\499320023.py:36: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


In [29]:
plt.imshow(dp_crop_dat[50,50,:,:])